In [ ]:
using Revise
using Paint

In [ ]:
using Serialization
using Images, ImageShow
using Plots
using Evolutionary
using BlackBoxOptim
using StaticArrays

In [ ]:
target = float.(load("lisa.png"))

In [ ]:
hist = Serialization.deserialize("output/simresult/simlog_100-prims_10x-blackbox_5000-batch_5-epoch_250-refine.bin")
nothing

In [ ]:
function drawstate(state, size)
    initial = zeros(RGB{Float32}, size) .+ state.background
    for k in eachindex(state.shapes)
        draw!(initial, state.shapes[k], state.current_colours[k], RasterAlgorithmScanline())
    end
    initial
end

In [ ]:
gr()
plots = []
anim_singles = @animate for i = 1:length(hist.history)+100
    ival = min(i, length(hist.history))
    res = hist.history[ival]
    drawsize = (200, 200)
    img = drawstate(res, drawsize)
    push!(plots, plot(img, size=drawsize, showaxis=true, ticks=false, title=string(ival-1)))
end
gif(anim_singles, fps=12)

In [ ]:
losses::Vector{Float32} = [state.best for state in hist.history]

In [ ]:
plotly()
plot(losses)

In [ ]:
function sample_loss(xs)
    tri = Triangle(SVector{6, Float32}(xs))
    col = averagepixel(target, tri, RasterAlgorithmScanline())
    loss = drawloss(target, hist.history[end].current, tri, col, SELoss(), RasterAlgorithmScanline())
    Float64(loss)
end

In [ ]:
optres = compare_optimizers(x -> sample_loss(x); SearchRange=(0, 1), NumDimensions=6, MaxTime=3, PopulationSize=1000)

In [ ]:
optres = bboptimize(x -> sample_loss(x), [0.48461180779532725, 0.2759816866858497, 0.6661557722723, 0.40521914605484205, 0.5720804506313976, 0.2886744590110042]; SearchRange=(0, 1), NumDimensions=6, MaxTime=10)

In [ ]:
nbatch = 100000
nepochs = 3
nrefinement = 100
@time minloss, mintri, mincol = simulate_iter_ga(hist.history[end], target, nbatch, nepochs, nrefinement, losstype=SELoss())

In [ ]:
img = copy(hist.history[100].current)
mintri = Triangle(SVector{6, Float32}([0.481231, 0.273921, 0.667457, 0.412713, 0.588618, 0.29568]))
draw!(img, mintri, averagepixel(target, mintri, RasterAlgorithmScanline()), RasterAlgorithmScanline())
# draw!(img, mintri, RGB{Float32}(1,0,1), RasterAlgorithmScanline())
img

In [ ]:
diff = Gray.(abs.(img .- target))
diff = diff ./ maximum(diff)

In [ ]:
blurdiff = imfilter(diff, Kernel.box((9,9)))

In [ ]:
perm = reverse(sortperm([blurdiff...]))
points = perm[1:1000]

In [ ]:
plt = plot(diff)
scatter!(plt, [swap(CartesianIndices(blurdiff)[pt].I) for pt in points])

In [ ]:
CartesianIndices(diff)[29227]

In [ ]:
swap(p) = p[2], p[1]

In [ ]:
el = transpose(reshape(reverse(blurdiff[sortperm([blurdiff...])]), (200, 200)))

In [ ]:
el ./ maximum(el)